In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.metrics import ndcg_score
tqdm.pandas()

In [2]:
size = 'demo'
type_ = 'validation'
amount = 1000
predictions_df_path = f'./files/pickle/predictions_df_{size}_{type_}_{str(amount)}.pkl'
predictions_df = pd.read_pickle(predictions_df_path)
print('Predictions df shape:                      ',predictions_df.shape)

Predictions df shape:                       (1000, 6)


In [3]:
predictions_df.head(2)

,user_id,article_ids_inview,article_ids_clicked,Predicted_read_times,Predicted_tuples_sorted,Predicted_article_ids
0,76658,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],"[159.59189, 118.70199, 17.448689, 41.142696, 1...","[(9788239, 159.59189), (9780702, 118.70199), (...","[9788239, 9780702, 9787499, 9553264, 9783042, ..."
1,76658,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],"[18.174896, 41.932983, 17.448689, 11.244547, 2...","[(9788125, 222.22842), (9786217, 41.932983), (...","[9788125, 9786217, 9788352, 9788521, 9553264, ..."


In [5]:
predictions_df['user_id'].value_counts()

user_id
1412107    25
1279122    19
1218901    18
790962     17
1230526    13
           ..
2015664     1
1286303     1
2118743     1
1680426     1
1321771     1
Name: count, Length: 366, dtype: int64

In [6]:
def get_reciprocal_rank(row):
    predicted_item_list = row['Predicted_article_ids']
    clicked_article = row['article_ids_clicked'][0]
    try:
        index = predicted_item_list.index(clicked_article)
        # Return the reciprocal rank
        return 1 / (index + 1)
    except ValueError:
        # If the clicked article is not in the predicted list, return 0
        return 0
    
def calculate_precision(target, predictions):
    tp = predictions.count(target)  # Count true positives
    fp = len(predictions) - tp  # Calculate false positives
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0  # Compute precision
    return precision

def calculate_recall(target, predictions):
    tp = predictions.count(target)  # True Positives: target in predictions
    fn = 1 if tp == 0 else 0  # False Negatives: target not in predictions
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0  # Compute recall
    return recall

In [7]:
predictions_df['MMR_rank'] = predictions_df.progress_apply(get_reciprocal_rank,axis=1)
predictions_df['Precision@10']= predictions_df.progress_apply(lambda row: calculate_precision(row['article_ids_clicked'][0],row['Predicted_article_ids']),axis=1)
predictions_df['Recall@10']= predictions_df.progress_apply(lambda row: calculate_recall(row['article_ids_clicked'][0],row['Predicted_article_ids']),axis=1)
predictions_df.head(2)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 25638.50it/s]


,user_id,article_ids_inview,article_ids_clicked,Predicted_read_times,Predicted_tuples_sorted,Predicted_article_ids,MMR_rank,Precision@10,Recall@10
0,76658,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],"[159.59189, 118.70199, 17.448689, 41.142696, 1...","[(9788239, 159.59189), (9780702, 118.70199), (...","[9788239, 9780702, 9787499, 9553264, 9783042, ...",0.2,0.166667,1.0
1,76658,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],"[18.174896, 41.932983, 17.448689, 11.244547, 2...","[(9788125, 222.22842), (9786217, 41.932983), (...","[9788125, 9786217, 9788352, 9788521, 9553264, ...",1.0,0.142857,1.0


In [8]:
predictions_df.shape

(1000, 9)

In [37]:
predictions_df[predictions_df['MMR_rank']==0]

,user_id,article_ids_inview,article_ids_clicked,Predicted_read_times,Predicted_tuples_sorted,Predicted_article_ids,MMR_rank,Precision@10,Recall@10
6,1014041,"[9277736, 9782449, 9783754, 9186608, 9279095, ...",[9780373],"[15.663772, 17.538553, 15.238331, 15.724276, 1...","[(9142564, 19.947956), (9782449, 17.538553), (...","[9142564, 9782449, 9784094, 9279095, 9186608, ...",0.0,0.0,0.0
13,1245658,"[9781467, 9779122, 9780280, 9754241, 9772869, ...",[9781467],"[13.179144, 17.469442, 18.163853, 22.165958, 1...","[(9781656, 27.77744), (9781476, 22.916412), (9...","[9781656, 9781476, 9754241, 9775964, 9776420, ...",0.0,0.0,0.0
14,1245658,"[9781855, 9142564, 9779653, 9551777, 9781872, ...",[9781855],"[13.73581, 14.840741, 17.004566, 13.683578, 12...","[(9781814, 29.442497), (9780919, 21.051601), (...","[9781814, 9780919, 9781856, 9781838, 9781621, ...",0.0,0.0,0.0
19,1412107,"[9774527, 9774352, 9779430, 9779956, 9777156]",[9779956],"[17.4634, 18.066206, 17.49035]","[(9774352, 18.066206), (9779430, 17.49035), (9...","[9774352, 9779430, 9774527]",0.0,0.0,0.0
23,1753136,"[9778939, 9780210, 9695098, 9780648, 9778381, ...",[9779288],"[21.637909, 14.70904, 22.966866, 20.626793, 17...","[(7184889, 30.72874), (9695098, 22.966866), (9...","[7184889, 9695098, 9780498, 9780547, 9778939, ...",0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
956,1885147,"[9782115, 9781502, 9781796, 9781887, 9695098, ...",[9777199],"[23.175953, 22.809563, 29.872189, 23.602043, 2...","[(9779659, 33.333107), (9781796, 29.872189), (...","[9779659, 9781796, 9779092, 9695098, 9277339, ...",0.0,0.0,0.0
973,1218901,"[8054212, 8514670, 9486080, 9455386, 9784782]",[9784782],"[21.959156, 21.656723, 21.179585, 23.554184]","[(9455386, 23.554184), (8054212, 21.959156), (...","[9455386, 8054212, 8514670, 9486080]",0.0,0.0,0.0
976,1218901,"[9455386, 8054212, 8514670, 9486080, 9784575, ...",[9784662],"[21.179585, 21.959156, 21.656723, 31.141968, 2...","[(9486080, 31.141968), (8054212, 21.959156), (...","[9486080, 8054212, 9784575, 8514670, 9455386]",0.0,0.0,0.0
977,1881817,"[9783886, 9780860, 9339920, 9777200, 9783729, ...",[9771113],"[16.708488, 17.711182, 17.933645, 17.130358, 1...","[(9779370, 22.120888), (9784444, 18.864944), (...","[9779370, 9784444, 9782763, 9784228, 9782126, ...",0.0,0.0,0.0


In [12]:
predictions_df_per_user = predictions_df.groupby('user_id')[['MMR_rank', 'Precision@10', 'Recall@10']].mean()

In [13]:
predictions_df_per_user.head(2)

,MMR_rank,Precision@10,Recall@10
user_id,,,
19181,0.612500,0.144742,1.0
22895,0.333333,0.111111,1.0


In [18]:
mrr_10 = predictions_df_per_user['MMR_rank'].mean()
precision_10 = predictions_df_per_user['Precision@10'].mean()
recall_10 = predictions_df_per_user['Recall@10'].mean()
f1_score_10 = (2*precision_10*recall_10)/ (precision_10 + recall_10)

print("MRR@10: ", mrr_10)
print("Precision@10: ", precision_10)
print("Recall@10: ", recall_10)
print("F1_Score@10: ", f1_score_10)

MRR@10:  0.3679819876049919
Precision@10:  0.12084973010931081
Recall@10:  0.8247199021050718
F1_Score@10:  0.21080875313596853


### Interpretation-Dropout

- An MRR of approximately 0.3103 means that, on average, the true positive item is found at about the third position in the recommendation list (since 1/0.299≈3.34). This is a reasonably good result, indicating that the recommender system often ranks the relevant items near the top of the recommendation list.

- A Precision@10 of approximately 0.119 means that about 12.8% of the items in the top 10 recommendations are relevant. This indicates that for every 10 items recommended, around 1.28 items are relevant. This precision value suggests that there is room for improvement in terms of recommending more relevant items in the top 10.

- A Recall@10 of 0.80 means that the recommender system successfully identifies 86% of the relevant items within the top 10 recommendations. This high recall value indicates that the system is very effective at finding relevant items, though they may not always be ranked at the very top of the list.

### Interpretation-OuterJoin

- An MRR of approximately 0.3102 means that, on average, the true positive item is found at about the third position in the recommendation list (since 1/0.299≈3.34). This is a reasonably good result, indicating that the recommender system often ranks the relevant items near the top of the recommendation list.

- A Precision@10 of approximately 0.118 means that about 12.8% of the items in the top 10 recommendations are relevant. This indicates that for every 10 items recommended, around 1.28 items are relevant. This precision value suggests that there is room for improvement in terms of recommending more relevant items in the top 10.

- A Recall@10 of 0.79 means that the recommender system successfully identifies 86% of the relevant items within the top 10 recommendations. This high recall value indicates that the system is very effective at finding relevant items, though they may not always be ranked at the very top of the list.

### Interpretation

- An MRR of approximately 0.299 means that, on average, the true positive item is found at about the third position in the recommendation list (since 1/0.299≈3.34). This is a reasonably good result, indicating that the recommender system often ranks the relevant items near the top of the recommendation list.

- A Precision@10 of approximately 0.128 means that about 12.8% of the items in the top 10 recommendations are relevant. This indicates that for every 10 items recommended, around 1.28 items are relevant. This precision value suggests that there is room for improvement in terms of recommending more relevant items in the top 10.

- A Recall@10 of 0.86 means that the recommender system successfully identifies 86% of the relevant items within the top 10 recommendations. This high recall value indicates that the system is very effective at finding relevant items, though they may not always be ranked at the very top of the list.

### Overall Interpretation
- MRR (0.299): The relevant items are generally ranked around the third position on average.
- Precision@10 (0.128): About 12.8% of the top 10 recommendations are relevant, indicating room for improvement in the quality of top recommendations.
- Recall@10 (0.86): The system successfully finds 86% of the relevant items within the top 10 recommendations, showing strong recall performance.

To improve precision and MRR for your recommender system, you can consider various strategies that involve improving your preprocessing, model, and recommendation logic. Here are some approaches:

1. Data Preprocessing Improvements
    1. Feature Engineering:

        - Add More Features: Incorporate additional features that could influence reading time predictions, such as article length, topic, author, publication date, user preferences, etc.
        - Normalization and Scaling: Ensure that your features are properly normalized and scaled to help the model learn more effectively.
        - Categorical Features: Use techniques like one-hot encoding or embeddings for categorical features (e.g., article categories or authors).

    2. Data Cleaning:

        - Remove Outliers: Identify and remove or handle outliers in reading times to ensure the model isn’t biased by extreme values.
        - Handle Missing Values: Ensure any missing values in the dataset are properly handled through imputation or removal.

    3. Data Augmentation:

        - Synthetic Data: If your dataset is small, consider generating synthetic data to improve model training.

2. Model Improvements

    1. Model Architecture:

        - Experiment with Different Architectures: Try different neural network architectures such as deeper networks, recurrent neural networks (RNNs), transformers, etc.
    - Hyperparameter Tuning: Perform hyperparameter tuning to find the optimal parameters for your model (learning rate, batch size, number of layers, units per layer, etc.).

    2. Training Techniques:

        - Regularization: Use regularization techniques like dropout, L2 regularization, and early stopping to prevent overfitting.
        - Ensemble Methods: Combine predictions from multiple models using ensemble methods (e.g., bagging, boosting) to improve accuracy and robustness.

3. Recommendation Logic Improvements

    1. Post-Processing Predictions:

        - Re-Ranking: After predicting reading times, re-rank articles using additional criteria such as user preferences, recent trends, or article popularity.
        - Hybrid Recommendations:

        - Combine Models: Use a hybrid approach that combines collaborative filtering, content-based filtering, and your read time prediction model to recommend articles.

In [11]:

### Feature engineering
## 1. Outlier Detection - Trained with read time <60 no such improvement 
## 2. Scroll percentage * read time-zero percentage what to do (NaN-not a good idea)- Not improved
## 3. Normalize-scaling 
## 4. Threshold
## 5. Outer join - 
## 6. Check for multiple articles for the same user



### Model Architecture
## 1. Learning Rate
## 2. Early Stopping
## 3. Dropout
## 4. Add more Dense,Layers
## 5. Timestamps-LSTM

### Hybrid Approach
## 1. Contend Based 

### Recommendations
## 1. Apply clustering

### Evaluation
## 1.Different Metric-F1

#MAke the right adjustments to make it run all at once

In [20]:
## Tests
#1. Outter join-Scroll percentage-> Not improved-Scrolled percentage not a good feature for the model in our test
#2. Outer join-Worse results
#3. Model Architecture-Dropout
#4. Model Architecture-EarlyStopping
